In [ ]:

#comment if not running on colab
from google.colab import drive
drive.mount('/content/drive')

In [1]:
#import libraries
import pandas as pd
import os
import numpy as np
import collections

In [10]:
#change project folder to match yours
#project_folder ='/content/drive/MyDrive/BioinfoProject/Project_Bioinformatics/First_project/'
project_folder ='/'

In [11]:
#change dataset folder to match yours
dataset_folder = os.path.join(project_folder,'dataset/')

In [2]:
#only in local
#dataset_folder="dataset/"

# 1.1

In [3]:
#download curated gene-diesase associtaions from https://www.disgenet.org/downloads and covnert tsv in csv

data = pd.read_csv(dataset_folder+ '1.1/curated_gene_disease_associations.csv')
#get only data for intereted gene id C0043094
gene_disease = data.loc[data['diseaseId'] == 'C0043094']

#get list og gene symbols
list_geneSymbol = gene_disease['geneSymbol']


#download hgnc dataset from https://www.genenames.org/download/statistics-and-files/
hgnc = pd.read_csv(dataset_folder+ "1.1/hgnc_complete_set.txt", sep='\t', dtype = 'unicode')

#get data (symbol,name,entrez_id,unirpot_idasdn status) for only genes stored in previous list
genes_diseases_hgnc = hgnc.loc[hgnc['symbol'].isin(list_geneSymbol)][['symbol','name','entrez_id','uniprot_ids','status']]

#get only approved data
genes_diseases_hgnc_app = genes_diseases_hgnc.loc[genes_diseases_hgnc['status']=='Approved']


#get list ofuniprotidthat we'll useto find gene descriptionin Uniprot
list_uniprotID = genes_diseases_hgnc_app['uniprot_ids']

#write in a file alll uniprot ids. This is necessary because we'll download only necessary description for these genes from uniprot 
with open(dataset_folder+ '1.1/uniprot_ids.txt', 'w') as f:
    for item in list_uniprotID:
        f.write("%s\n" % item)
    f.close()

#download data containing description for every gene. We used txt file for download them
gene_description = pd.read_csv(dataset_folder+ '1.1/uniprot-yourlist%3AM20201123A94466D2655679D1FD8953E075198DA817123DO.csv')

#rename column for the function of the gene
gene_description=gene_description.rename(columns={"yourlist:M20201123A94466D2655679D1FD8953E075198DA817123DO":"uniprot_ids"})

#merge two dataset. One containing all info about gene and the second one conatining description about their functions
final_genes_deseas = pd.merge(gene_description, genes_diseases_hgnc_app, on='uniprot_ids')
#rename column
final_genes_deseas=final_genes_deseas.rename(columns={"Function [CC]":"function"})
#get only first part of gene description
final_genes_deseas['Description']= final_genes_deseas.function.str[10:150]
#delete not necessary columns
final_genes_deseas = final_genes_deseas.drop(['function','Entry','status'], axis = 1) 
final_genes_deseas = final_genes_deseas[['symbol','uniprot_ids','name','entrez_id','Description']]

In [4]:
#fix column names and save xlsx file
final_genes_deseas.rename(columns={"symbol":"Gene symbol","uniprot_ids":"Uniprot AC","name":'Protein name','entrez_id':"Entrez Gene ID"})
final_genes_deseas.to_excel(dataset_folder+ '1.1/1.1-gene_deseas_associations.xlsx')
final_genes_deseas.to_csv(dataset_folder+ '1.1/1.1-gene_deseas_associations.csv')

In [5]:
final_genes_deseas

,symbol,uniprot_ids,name,entrez_id,Description
0,ABCG1,P45844,ATP binding cassette subfamily G member 1,9619,Catalyzes the efflux of phospholipids such as ...
1,ACADM,P11310,acyl-CoA dehydrogenase medium chain,34,Acyl-CoA dehydrogenase specific for acyl chain...
2,ACE,P12821,angiotensin I converting enzyme,1636,Converts angiotensin I to angiotensin II by re...
3,ADIPOQ,Q15848,"adiponectin, C1Q and collagen domain containing",9370,Important adipokine involved in the control of...
4,AHR,P35869,aryl hydrocarbon receptor,196,Ligand-activated transcriptional activator. Bi...
...,...,...,...,...,...
97,TSEN2,Q8NCE0,tRNA splicing endonuclease subunit 2,80746,Constitutes one of the two catalytic subunit o...
98,VIM,P08670,vimentin,7431,Vimentins are class-III intermediate filaments...
99,VPS13D,Q5THJ4,vacuolar protein sorting 13 homolog D,55187,Functions in promoting mitochondrial clearance...
100,ZDHHC2,Q9UIJ5,zinc finger DHHC-type palmitoyltransferase 2,51201,Palmitoyltransferase that catalyzes the additi...


# 1.2

# Get biogrid data

In [13]:
#read biogrid dataset 
biogrid_datafile = dataset_folder+'1.2/BIOGRID-ORGANISM-Homo_sapiens-4.2.191.tab3.txt'
biogrid_data = pd.read_csv(biogrid_datafile,sep='\t')
#biogrid_data.info()

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (1,18) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [21]:
biogrid_data.head(5)

,#BioGRID Interaction ID,Entrez Gene Interactor A,Entrez Gene Interactor B,BioGRID ID Interactor A,BioGRID ID Interactor B,Systematic Name Interactor A,Systematic Name Interactor B,Official Symbol Interactor A,Official Symbol Interactor B,Synonyms Interactor A,...,TREMBL Accessions Interactor B,REFSEQ Accessions Interactor B,Ontology Term IDs,Ontology Term Names,Ontology Term Categories,Ontology Term Qualifier IDs,Ontology Term Qualifier Names,Ontology Term Types,Organism Name Interactor A,Organism Name Interactor B
0,103,6416,2318,112315,108607,-,-,MAP2K4,FLNC,JNKK|JNKK1|MAPKK4|MEK4|MKK4|PRKMK4|SAPKK-1|SAP...,...,Q59H94,NP_001120959|NP_001449,-,-,-,-,-,-,Homo sapiens,Homo sapiens
1,117,84665,88,124185,106603,-,-,MYPN,ACTN2,CMD1DD|CMH22|MYOP|RCM4,...,Q59FD9|F6THM6,NP_001094|NP_001265272|NP_001265273,-,-,-,-,-,-,Homo sapiens,Homo sapiens
2,183,90,2339,106605,108625,-,-,ACVR1,FNTA,ACTRI|ACVR1A|ACVRLK2|ALK2|FOP|SKR1|TSRI,...,-,NP_002018,-,-,-,-,-,-,Homo sapiens,Homo sapiens
3,278,2624,5371,108894,111384,-,-,GATA2,PML,DCML|IMD21|MONOMAC|NFE1B,...,-,NP_150250|NP_150253|NP_150252|NP_150247|NP_150...,-,-,-,-,-,-,Homo sapiens,Homo sapiens
4,418,6118,6774,112038,112651,RP4-547C9.3,-,RPA2,STAT3,REPA2|RP-A p32|RP-A p34|RPA32,...,-,NP_644805|NP_003141|NP_001356447|NP_001356443|...,-,-,-,-,-,-,Homo sapiens,Homo sapiens


In [22]:
gene_A_list = biogrid_data.iloc[:,1].values
gene_B_list = biogrid_data.iloc[:,2].values
A_B_list = np.append(gene_A_list,gene_B_list)
A_set = set(gene_A_list)
len(A_set)


20765

In [23]:
B_set =  set(gene_B_list)
len(B_set)

23834

In [24]:
A_B_set = set(A_B_list)
len(A_B_set)

29449

#Get all nodes involved in interactions 
#removing non numeric nodes

---



In [25]:
genes_biogrid = np.array([x for x in A_B_set if str.isdigit(str(x))])
genes_biogrid

array(['1', '2', '131076', ..., '115209', '2101', '23383'], dtype='<U21')

#Get all interactions 
#removing invalid nodes with invalid ID 


In [26]:
interactions = biogrid_data.iloc[:,1:3].values
#interactions =np.array([[a,b] for a,b in interactions if(str.isdigit(str(a)) & str.isdigit(str(b)))])
interactions =np.array([[a,b] for a,b in interactions ])
interactions.shape

(697802, 2)

# Seed gene list 

In [27]:
seed_genes_ids  =  final_genes_deseas['entrez_id'].values
seed_genes_ids

array(['9619', '34', '1636', '9370', '196', '1646', '302', '308', '323',
       '351', '23365', '586', '801', '857', '284119', '948', '25932',
       '1268', '26958', '1509', '1565', '8701', '1813', '1783', '2146',
       '285172', '2199', '23014', '9637', '2246', '55277', '11167',
       '79068', '9518', '2688', '51738', '2784', '2888', '2934', '84064',
       '3358', '3620', '3488', '51141', '51194', '8826', '3778', '3779',
       '9215', '3929', '3952', '3953', '9926', '4134', '22919', '4137',
       '4199', '4232', '284207', '4313', '4321', '23164', '4430', '4651',
       '55706', '4715', '140609', '4811', '4883', '114882', '5020',
       '23022', '51449', '5290', '5443', '5467', '23683', '5631', '29108',
       '10266', '5986', '6158', '6202', '6234', '6259', '51091', '91404',
       '26040', '26037', '10569', '8470', '6678', '6711', '57620', '412',
       '7124', '7171', '80746', '7431', '55187', '51201', '55055'],
      dtype=object)

#Now we build a dictionary of all interactions

In [28]:
all_interactions_dict = collections.defaultdict(set)
for a,b in interactions:
  all_interactions_dict[a].add(b)
  all_interactions_dict[b].add(a)

# Interactions including seed genes 

In [29]:
seed_interactions_dict = collections.defaultdict(set)
for key in seed_genes_ids:
  seed_interactions_dict[key] = all_interactions_dict[key]

# Protein interacting with seed genes

In [30]:
non_seed_genes_ids = set()
# from seed genes interactions dictionary, we retrieve all genes interacting with them
# performing union of all set 
for key in seed_interactions_dict.keys():
  non_seed_genes_ids = non_seed_genes_ids.union(seed_interactions_dict[key]) 

#We remove all seed genes included in the set
non_seed_genes_ids = np.array([x for x in non_seed_genes_ids if x not in seed_genes_ids])

# There we get only non seed gene which interacted with at least one seed gene
len(non_seed_genes_ids)

6140

#Then we create a dictionary for interactions of non seed genes involved with at least one seed gene

In [31]:
non_seed_interactions_dict = collections.defaultdict(set)

#looping on non seed gend ids 
for key in non_seed_genes_ids:
  # we retrieve interactions from all interactions dictionary 
  related_genes = all_interactions_dict[key]

  related_non_seed_genes = [x for x in related_genes if x not in seed_genes_ids]

  #we add it tho the seed genes iteractions dictionary 
  non_seed_interactions_dict[key] = set(related_non_seed_genes)


#Lets put all this in a data frame

In [32]:
values = []
for a ,b in seed_interactions_dict.items():
  values.append([a,'T',','.join(b)])

for a ,b in non_seed_interactions_dict.items():
  values.append([a,'F',','.join(b)])


df = pd.DataFrame(values,columns=['gene_id','seed_gene_involved','interactions_seeds_ids'])

In [33]:
df.sample(10)

,gene_id,seed_gene_involved,interactions_seeds_ids
1218,257629,F,"10083,9258,5339,120892,57154"
1861,238,F,"10188,3009,10399,1025,142679,53358,5335,2064,3..."
1708,114880,F,"8844,112399,49854,6242,55667,10959,51133,57622..."
10,23365,T,"7706,5792,387,55789,115708,4646,10672,27252,10..."
5511,18053,F,2885
1722,48,F,"4703,113246,10482,84817,4343,856924,1540,14896..."
2637,733686,F,54627
4748,5469,F,"9439,23476,9412,547,29079,80854,9150,55671,857..."
3976,9939,F,"65110,22928,9055,4799,3921,7919,9675,155871,55..."
3175,132141,F,"22884,817,441250,55034,22894,5824,10381,8458,2..."


#1.2 b - Store data in csv format

In [34]:
filename = dataset_folder + '1.2/interactions.csv'
df.to_csv(filename, sep='\t',index=False)

#1.2 c - Summary

In [35]:
n_seed_disgenet = len(set(seed_genes_ids))
n_seed_biogrid = len(set([gene for gene in genes_biogrid if gene in seed_genes_ids]))
n_interacting_genes = len(set(genes_biogrid))
n_interactions = biogrid_data.shape[0]

In [36]:
table = [[n_seed_disgenet,n_seed_biogrid,n_interacting_genes,n_interactions]]
summary = pd.DataFrame(table, columns=['N° seed disgenet','N° seed biogrid','Interacting genes','Interactions'])
summary

,N° seed disgenet,N° seed biogrid,Interacting genes,Interactions
0,102,101,26221,697802


#1.3 Arrange interaction data

In [37]:
genes_interactions = biogrid_data.values
indexes = []
i =0
for row in genes_interactions:
  check = str(row[1]) in seed_genes_ids
  check_2 = str(row[2]) in seed_genes_ids
  if check_2  and check:
    indexes.append(i)
    i= i+1
  



#1.2 a - seed genes interactome: interactions that involve seed genes only

In [38]:
only_seed_interactions = biogrid_data.iloc[indexes,[7,8,23,26]]
only_seed_data = only_seed_interactions.values
columns = ['interactor A gene symbol',' interactor B gene symbol', 'interactor A Uniprot AC', 'interactor B Uniprot AC']
only_seed_interactions_df = pd.DataFrame(only_seed_data,columns=columns)

In [39]:
#save gene seed interactions into csv file
filename = dataset_folder + '1.3/gsi.csv'
only_seed_interactions_df.to_csv(filename, sep='\t',index=False)


#1.2 All proteins interacting with at least one seed gene

In [40]:
proteins_interactions = biogrid_data.values
indexes = []
i =0
for row in proteins_interactions:
  check_1 = str(row[1]) in seed_genes_ids
  check_2 = str(row[2]) in seed_genes_ids
  check_3 = row[35] =='Homo sapiens'
  check_4 = row[36] =='Homo sapiens'
  if (check_2  or check_1) & check_3 & check_4:
    indexes.append(i)
    i= i+1

In [41]:
all_proteins_interactions = biogrid_data.iloc[indexes,[7,8,23,26]]
all_proteins_data = all_proteins_interactions.values
columns = ['interactor A gene symbol',' interactor B gene symbol', 'interactor A Uniprot AC', 'interactor B Uniprot AC']
all_proteins_interactions_df = pd.DataFrame(all_proteins_data,columns=columns)

In [42]:
#save inteactome disease csv
filename = dataset_folder + '1.3/interactome.csv'
all_proteins_interactions_df.to_csv(filename, sep='\t',index=False)

